<a href="https://colab.research.google.com/github/samuelgamaral/INFO_NFS/blob/main/INFO_NFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

!git clone https://github.com/samuelgamaral/INFO_NFS.git

# Função para extrair dados de um XML de NFS-e
def extrair_dados_nfse(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    ns = {'ns': root.tag.split('}')[0].strip('{')}  # Detecta o namespace automaticamente

    try:
        prestador_cnpj = root.find('.//ns:PrestadorServico/ns:IdentificacaoPrestador/ns:Cnpj', ns).text
        tomador_cnpj = root.find('.//ns:TomadorServico/ns:IdentificacaoTomador/ns:CpfCnpj/ns:Cnpj', ns)
        if tomador_cnpj is None:
            tomador_cnpj = root.find('.//ns:TomadorServico/ns:IdentificacaoTomador/ns:CpfCnpj/ns:Cpf', ns)
        tomador_cnpj = tomador_cnpj.text if tomador_cnpj is not None else None

        valor_servico = root.find('.//ns:Valores/ns:ValorServicos', ns).text
        valor_iss = root.find('.//ns:Valores/ns:ValorIss', ns)
        data_emissao = root.find('.//ns:DataEmissao', ns).text
        numero_nf = root.find('.//ns:IdentificacaoNfse/ns:Numero', ns)

        return {
            'arquivo': os.path.basename(xml_path),
            'numero_nf': numero_nf.text if numero_nf is not None else 'N/A',
            'prestador_cnpj': prestador_cnpj,
            'tomador_cnpj': tomador_cnpj,
            'valor_servico': float(valor_servico),
            'valor_iss': float(valor_iss.text) if valor_iss is not None else 0.0,
            'data_emissao': data_emissao
        }

    except Exception as e:
        return {'erro': str(e), 'arquivo': os.path.basename(xml_path)}

# Pasta com os arquivos XML
pasta_xml = '/content/INFO_NFS'  # Altere para o caminho correto

dados_nfse = []

# Processa todos os arquivos XML da pasta
for arquivo in os.listdir(pasta_xml):
    if arquivo.endswith('.xml'):
        caminho = os.path.join(pasta_xml, arquivo)
        dados_nfse.append(extrair_dados_nfse(caminho))

# Cria DataFrame
df = pd.DataFrame(dados_nfse)

# Converte data
df['data_emissao'] = pd.to_datetime(df['data_emissao'], errors='coerce').dt.tz_localize(None)

# Verificações de inconsistência
df['valor_servico_invalido'] = df['valor_servico'] <= 0
df['data_futura'] = df['data_emissao'] > pd.Timestamp.now()
df['cnpj_tomador_faltando'] = df['tomador_cnpj'].isnull()

# Exporta para Excel
df.to_excel('relatorio_nfse_completo.xlsx', index=False)

# Retorno
print("Resumo gerado com sucesso!")


fatal: destination path 'INFO_NFS' already exists and is not an empty directory.
Resumo gerado com sucesso!
